In [29]:
"""Example of a simple nurse scheduling problem."""
from ortools.sat.python import cp_model

#User interface : 
#https://docs.google.com/spreadsheets/d/1bdXz3_N18NCq1F3Zk1kiqDSAH_-dRc87yXG5IZCFtLc/edit?usp=sharing

def Ask_nurse_name():
    num_nurses = int(input("Enter number of nurses :"))
    all_nurses = range(num_nurses)
    nurse_names = []
    for n in all_nurses:
        nurse_names.append(input("Enter name of nurse number " + str(n+1) + " :"))
    return nurse_names

def Ask_nurse_holidays(nurse_names):
    nurse_holidays = []
    all_nurses_holidays = range(len(nurse_names))
    for n in all_nurses_holidays:
        nurse_holidays.append(input("Enter number of days off for nurse " + str(nurse_names[n]) + " :"))
    return nurse_holidays

def Ask_nurse_rank(nurse_names):
    nurse_ranks = []
    all_nurses_ranks = range(len(nurse_names))
    for n in all_nurses_ranks:
        nurse_ranks.append(input("Enter rank for nurse " + str(nurse_names[n]) + " :"))
    return nurse_ranks

def Calculated_adjusted_ranks(nurse_rank, nurse_holidays, num_days):
    adjusted_rank = [] #ranks adjusted to number of working days of each nurse
    nurse_range = range(len(nurse_rank))
    for n in nurse_range:
        adjusted_rank.append(int(nurse_rank[n])*(num_days-int(nurse_holidays[n]))/num_days)
        print("Adjusted rank number " + str(n+1) + " : " + str(adjusted_rank[n]))     
    return adjusted_rank


def Calculate_nurse_shifts(adjusted_ranks, num_shifts, num_days):
    shift_per_nurse = []
    total_rank = 0
    total_shift_rounded = 0                        
    all_nurses_shifts = range(len(adjusted_ranks))
    
    for n in all_nurses_shifts:
        total_rank = total_rank + adjusted_ranks[n]
        
    for n in all_nurses_shifts:    
        shift_per_nurse.append(round(num_shifts*num_days/total_rank*int(adjusted_ranks[n])))
        total_shift_rounded = total_shift_rounded + round(num_shifts*num_days/total_rank*int(adjusted_ranks[n]))
    
    shift_per_nurse[len(adjusted_ranks)-1] = round(num_shifts*num_days/total_rank*int(adjusted_ranks[len(adjusted_ranks)-1]))+num_shifts*num_days-total_shift_rounded
   
    print("Total rounded shifts : " + str(total_shift_rounded))                          
    print("Total rank = " + str(total_rank))
         
    return shift_per_nurse
    

def main():
    # Data.
    num_shifts = 1
    num_days = 29
    all_shifts = range(num_shifts)
    all_days = range(num_days)
    
    #Get nurse names
    nurse_names = []
    nurse_names = Ask_nurse_name()
    
    num_nurses = len(nurse_names)
    all_nurses = range(num_nurses)
    
    #Get nurse holidays
    nurse_holidays = []
    nurse_holidays = Ask_nurse_holidays(nurse_names)
    
    #Get nurse ranks
    nurse_rank = []
    nurse_rank = Ask_nurse_rank(nurse_names)
                             
    #Calculate ranks adjusted by days off
    adjusted_ranks = []
    adjusted_ranks = Calculated_adjusted_ranks(nurse_rank, nurse_holidays, num_days)
    
    #Calculate final nurse shifts based on ranks and days off
    # see https://docs.google.com/spreadsheets/d/1OsIMVJO8hlFbkvxDZ9C11IDY7806Zo2YwZpz5cG3Tzw/edit#gid=0    
    nurse_shifts = []
    nurse_shifts = Calculate_nurse_shifts(adjusted_ranks, num_shifts, num_days)
 

    # Creates the model.
    model = cp_model.CpModel()

    # Creates shift variables.
    # shifts[(n, d, s)]: nurse 'n' works shift 's' on day 'd'.
    shifts = {}
    for n in all_nurses:
        for d in all_days:
            for s in all_shifts:
                shifts[(n, d,
                        s)] = model.NewBoolVar('shift_n%id%is%i' % (n, d, s))

    # Each shift is assigned to exactly one nurse in the schedule period.
    for d in all_days:
        for s in all_shifts:
            model.Add(sum(shifts[(n, d, s)] for n in all_nurses) == 1)

    # Each nurse works at most one shift per day.
    for n in all_nurses:
        for d in all_days:
            model.Add(sum(shifts[(n, d, s)] for s in all_shifts) <= 1)
            
    #Shift distribution based on ranks 
    for n in all_nurses:
        print("Number of shifts for " + nurse_names[n] + " = " + str(nurse_shifts[n]))
        num_shifts_worked = []
        for d in all_days:
            for s in all_shifts:
                num_shifts_worked.append(shifts[(n, d, s)])
        model.Add(sum(num_shifts_worked) <= nurse_shifts[n])

    # Creates the solver and solve.
    solver = cp_model.CpSolver()
    solver.parameters.linearization_level = 0
    # Enumerate all solutions.
    solver.parameters.enumerate_all_solutions = True


    class NursesPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
        """Print intermediate solutions."""

        def __init__(self, shifts, num_nurses, num_days, num_shifts, limit):
            cp_model.CpSolverSolutionCallback.__init__(self)
            self._shifts = shifts
            self._num_nurses = num_nurses
            self._num_days = num_days
            self._num_shifts = num_shifts
            self._solution_count = 0
            self._solution_limit = limit

        def on_solution_callback(self):
            self._solution_count += 1
            print('Solution %i' % self._solution_count)
            for d in range(self._num_days):
                print('Day %i' % d)
                for n in range(self._num_nurses):
                    is_working = False
                    for s in range(self._num_shifts):
                        if self.Value(self._shifts[(n, d, s)]):
                            is_working = True
                            print('  Nurse %i works shift %i' % (n, s))
                    if not is_working:
                        print('  Nurse {} does not work'.format(n))
            if self._solution_count >= self._solution_limit:
                print('Stop search after %i solutions' % self._solution_limit)
                self.StopSearch()

        def solution_count(self):
            return self._solution_count

    # Display the first five solutions.
    solution_limit = 5
    solution_printer = NursesPartialSolutionPrinter(shifts, num_nurses,
                                                    num_days, num_shifts,
                                                    solution_limit)

    solver.Solve(model, solution_printer)

    # Statistics.
    print('\nStatistics')
    print('  - conflicts      : %i' % solver.NumConflicts())
    print('  - branches       : %i' % solver.NumBranches())
    print('  - wall time      : %f s' % solver.WallTime())
    print('  - solutions found: %i' % solution_printer.solution_count())


if __name__ == '__main__':
    main()

Enter number of nurses : 5
Enter name of nurse number 1 : Kent
Enter name of nurse number 2 : Dinç
Enter name of nurse number 3 : Serra
Enter name of nurse number 4 : Ceren
Enter name of nurse number 5 : Arin
Enter number of days off for nurse Kent : 0
Enter number of days off for nurse Dinç : 0
Enter number of days off for nurse Serra : 0
Enter number of days off for nurse Ceren : 0
Enter number of days off for nurse Arin : 20
Enter rank for nurse Kent : 1
Enter rank for nurse Dinç : 2
Enter rank for nurse Serra : 3
Enter rank for nurse Ceren : 4
Enter rank for nurse Arin : 5


Adjusted rank number 1 : 1.0
Adjusted rank number 2 : 2.0
Adjusted rank number 3 : 3.0
Adjusted rank number 4 : 4.0
Adjusted rank number 5 : 1.5517241379310345
Total rounded shifts : 29
Total rank = 11.551724137931034
Number of shifts for Kent = 3
Number of shifts for Dinç = 5
Number of shifts for Serra = 8
Number of shifts for Ceren = 10
Number of shifts for Arin = 3
Solution 1
Day 0
  Nurse 0 does not work
  Nurse 1 does not work
  Nurse 2 works shift 0
  Nurse 3 does not work
  Nurse 4 does not work
Day 1
  Nurse 0 works shift 0
  Nurse 1 does not work
  Nurse 2 does not work
  Nurse 3 does not work
  Nurse 4 does not work
Day 2
  Nurse 0 does not work
  Nurse 1 does not work
  Nurse 2 works shift 0
  Nurse 3 does not work
  Nurse 4 does not work
Day 3
  Nurse 0 does not work
  Nurse 1 does not work
  Nurse 2 works shift 0
  Nurse 3 does not work
  Nurse 4 does not work
Day 4
  Nurse 0 does not work
  Nurse 1 works shift 0
  Nurse 2 does not work
  Nurse 3 does not work
  Nurse 4 do

In [ ]:
he
